<h1><center>WORD EMBEDDINGS FOR TEXTUAL SIMILARITY</center></h1>
<h2><center>BIKASH GYAWALI</center></h2>
<h1><center>bikashg@live.com</center></h1>

<img src="wordEmbedding_IO.jpg">

<img src="word-embeddings.png">

In [ ]:
# Image copied from https://www.adityathakker.com/introduction-to-word2vec-how-it-works/

# 1. Word2Vec

In [ ]:
import gensim

import numpy as np

word2vecModel = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin',binary=True)

In [ ]:
vec_computer = word2vecModel.word_vec("computer")

In [ ]:
print(vec_computer)

In [ ]:
print(len(vec_computer))

In [ ]:
vec_website = word2vecModel.word_vec("website")

In [ ]:
print(len(vec_website))

In [ ]:
vec_orange = word2vecModel.word_vec("orange")

In [ ]:
print(len(vec_orange))

In [ ]:
vec_oov = word2vecModel.word_vec("embeddings")

## Compute similarity of words

The cosine similarity value can range from -1 (exactly opposite) to 1 (exactly the same). See https://en.wikipedia.org/wiki/Cosine_similarity

In [ ]:
similarity_computer_website = word2vecModel.similarity('computer', 'website')
print(similarity_computer_website)

In [ ]:
similarity_orange_website = word2vecModel.similarity('orange', 'website')
print(similarity_orange_website)

In [ ]:
def compute_cosine_similarity(a,b):
#     Copied from https://skipperkongen.dk/2018/09/19/cosine-similarity-in-python/
    dot = np.dot(a, b)
    norma = np.linalg.norm(a)
    normb = np.linalg.norm(b)
    cos = dot / (norma * normb)
    return cos



print(compute_cosine_similarity(vec_computer,vec_website))
print(compute_cosine_similarity(vec_orange,vec_website))

## Demo
http://bionlp-www.utu.fi/wv_demo/

# 2. BERT Embeddings

BERT is a newly released language understanding model from Google. For more details, see : https://github.com/google-research/bert. 

Among others, you can extract word embeeddings from the model. These are *contextualised* word embeddings.

https://github.com/imgarylai/bert-embedding

In [ ]:
from bert_embedding import BertEmbedding
import numpy as np
import string, random
import re

In [ ]:
bert_embedding = BertEmbedding()

In [ ]:
test_sentence1 = ["I bought an apple"]
# test_sentence2 = ["I bought an apple iphone"]
bert_vector1 = bert_embedding(test_sentence1)
print(bert_vector1)

In [ ]:
len(bert_vector1)

In [ ]:
bert_vector1[0][0]

In [ ]:
test_sentence1_apple_vector = bert_vector1[0][1][3]
test_sentence1_apple_vector

In [ ]:
print(len(bert_vector1[0][1][0])) # I
print(len(bert_vector1[0][1][1])) # bought
print(len(bert_vector1[0][1][2])) # an
print(len(bert_vector1[0][1][3])) # apple

In [ ]:
test_sentence2 = ["I bought an apple iphone"] # Previous sentence : I bought an apple.
bert_vector2 = bert_embedding(test_sentence2)

In [ ]:
test_sentence2_apple_vector = bert_vector2[0][1][3]
test_sentence2_apple_vector

In [ ]:
similarity_apples = compute_cosine_similarity(test_sentence1_apple_vector,test_sentence2_apple_vector)

# i bought an apple
# i bought an apple iphone

print(similarity_apples)

In [ ]:
# bought

similarity_bought = compute_cosine_similarity(bert_vector1[0][1][1],bert_vector2[0][1][1])

print(similarity_bought)

# OOV


### The BERT model we used was trained on the book corpus and the wikipedia corpus.

### BERT uses a WordPiece model -- if the complete word is not in its vocab, resort to subword. The lowest level of fallback is character.

### BERT vocab consists of : ~30,000 most common words and subwords in English + all characters 

In [ ]:
oov_sentence = ["I want to learn dlkfjegsdvkn"]
bert_oov = bert_embedding(oov_sentence)
print(bert_oov)

In [ ]:
bert_oov[0][1][4]

## Practical use case : Lets identify near-duplicate document pairs.

In [ ]:
def split2sentences(inputText):
    splits = re.split('\.|\n',inputText)  # Dot or a newline = sentence dimiliter. Sometimes, there is no space after dot for sentences due to malformed text.
    return list(filter(None, splits)) # remove all empty elements caused by repeated dimiliters

In [ ]:
def getVector(inputText, model=None):
    
    sentences = split2sentences(inputText)
    
    result = bert_embedding(sentences)  # We can pass the array of sentences all at once to get bert embeddings.
    
    
    avg_document_vector = np.zeros(shape=(len(sentences),768)) # Each BERT vector is of length 768
    count_sentences = 0
    
    
    for i in range(len(result)):
        current_result = result[i]
        
        current_sentence_tokens = current_result[0]
        current_sentence_vectors = current_result[1]
        
        # Sometimes, due to poor sentence identification, can end up with just one/few word sentences -- ignore.
        if len(current_sentence_tokens)<3 : 
            continue

        avg_current_sentence_vector = np.mean(current_sentence_vectors, axis=0) # to take the mean of each col across vectors for vectors obtained for each token.
          
        avg_document_vector[count_sentences] = avg_current_sentence_vector
        count_sentences = count_sentences + 1
        
    
    avg_document_vector = np.mean(avg_document_vector, axis=0) # to take the mean of each col across vectors for vectors obtained for each sentence.
    
    return avg_document_vector

In [ ]:
doc1 = '''In this paper we test for the existence of a long-run relationship between investment and savings (the Feldstein-Horioka puzzle) in a panel of 18 OECD countries, 1970-2007, allowing for heterogenous breaks in the coefficients. For this purpose we develop a bootstrap panel cointegration test with breaks robust to cross-section dependence. The test suggests that, even allowing for breaks in the countries where capital control regulations changed in the sample period, there is no evidence of an investment-savings long-run relationship for the panel as a whole.'''

In [ ]:
doc2 = '''In this paper we test for the existence of a long-run savings-investments relationship in 18 OECD economies over the period 1970-2007. Although individual modelling provides only very weak support to the hypothesis of a link between savings and investments, this cannot be ruled out as individual time series tests may have low power. We thus construct a new bootstrap test for panel cointegration robust to short- and long-run dependence across units. Thid test provides evidence of a long-run savings-investments relationship in about half of the OECD economies\nexamined. The elasticities are however often smaller than 1, the value expected under no capital\nmovements.'''

In [ ]:
bert_doc1 = getVector(doc1)
bert_doc2 = getVector(doc2)

In [ ]:
similarity_docs = compute_cosine_similarity(bert_doc1,bert_doc2)

print(similarity_docs)

## BERT in specific text domains.

Scientific Papers : SCIBERT (https://github.com/allenai/scibert)

Biomedical Text : BioBERT (https://github.com/dmis-lab/biobert)

Clinical Text : ClinicalBERT (https://github.com/kexinhuang12345/clinicalBERT)

# To summarise

BERT is : 
    
    Contextual
    OOV : Word Piece model.
    Higher feature dimensions
    Different neural architecture : (shallow vs more deep)
    Training dataset -- nature and size
    State of the art results 
    Adapted to more/many general/domain specific text



What was not discussed in this presentation :

    BERT is more than just word embeddings -- fine tune it for downstream NLP tasks.

In [ ]:

I saw a cat yesterday
0  1  2  3   4

     Input Feature    Label
ex1. 0 1                2
ex2. 1 2 .              3
ex3  2 3                4


      2 .           0 1



